# Getting Physical Compute Inventory from Intersight using the Cisco Intersight REST API with Python

In this lab you learn how to retrieve a list of physical compute inventory from Cisco Intersight using the Intersigight Python SDK.

## Objectives

The objective of this lab is to show how to:

* Authenticate with the Intersight using the Python SDK API's
* Call the Intersight API to pull a list of physical compute resources

## Prerequisites

Before getting started with this lab, you will need the following:

* An Intersight account (you may also need a Cisco account for credentials)
* API key
* API secret available 

### Background

Intersight has many capabilities and one of them is managing on-premises compute resources such as Cisco UCS servers. The process of making UCS servers available to Intersight is known as a claim. Thus, an administrator must first claim UCS servers before they are available as a resource in Intersight. After a UCS servers are claimed, they show up in inventory as physical compute servers.

In this lab we call the Intersight REST API's using Python and retrieve the inventory of physical compute resources claimed in Intersight.

## Step 1: Generate API key and Secret Key

Log into your Intersight account and navigate to **Settings** which is available by clicking the gear icon located towaward the upper-righthand side of the Intersight user interface.

<img align="left" src="images/intersight-settings.png">

* Click **API Keys** located in the lefthand column.
* Click **Generate API Key** located toward the upper-righthand side of the page.
* Select **API key for OpenAPI schema version 2** in the **Generate API Key** dialog box and provide a description for your API key.
* Copy the API Key ID and store it somewhere for use in the upcoming steps.
* Save the **Secret Key** to a text file and make note of its location. Move the file to the same directory where this code is running or make note of its location for use in the upcoming steps.

<img align="left" src="images/api-secret-key-save-as.png">

> This is the only one time that the secret key can be viewed or downloaded. You cannot recover them later. However, you can create new access keys at any time.

## Step 2: Import modules

Let's import `json` and `requests` as well as the `IntersightAuth` method from the `intersight_auth` class.

In [1]:
import json
import requests

from intersight_auth import IntersightAuth

## Step 3: Declare and populate a variable named AUTH

In step 2, you generated a new API Key ID and a Secret Key. Copy/paste the value of the `api_key_id` below as well as the file path for the downloaded file (also from Step 1) named `SecretKey.txt`.

In [2]:
AUTH = IntersightAuth(
    secret_key_filename='./key/SecretKey.txt',
    api_key_id='596cc79e5d91b400010d15ad/6037f58b7564612d33a5bb70/6139462a7564612d33532a71'
    )

## Step 4: Declare and populate a variable with the base URL
We'll use the base URL of `https://www.intersight.com/api/v1/` for this exercise.

In [3]:
BURL = 'https://www.intersight.com/api/v1/'

## Step 5: GET Phsyical Compute Summary

The GET operations provide insight into the hardware claimed by Intersight. In the example below, we'll get an inventory of compute nodes and see if they are managed by IMM (Intersight Managed Mode).

First, define the resource URI of `compute/PhysicalSummaries` and add it to the base url. Then, make the request  with the full URL along with the `auth` credentials and store Intersight's response in the variable named `response`. The request is in `try` and `except` blocks to handle exceptions.

In [4]:
resource_path = "compute/PhysicalSummaries"
# Let's add some error handling just in case
try:
    response = requests.get(
                    BURL + resource_path + '?$top=200',
                    auth=AUTH
                    )
    response.raise_for_status()

except requests.exceptoins.HTTPError as error:
    print(error) # Anything but a 200 response ends up here

## Step 6: Inspect the JSON returned by Intersight
Take a look at the `Results` data structure. There is a rich set of information returned by Intersight about the phsyical compute devices.

In [5]:
response.json()['Results']

[{'AccountMoid': '596cc79e5d91b400010d15ad',
  'AdminPowerState': 'policy',
  'AlarmSummary': {'ClassId': 'compute.AlarmSummary',
   'Critical': 1,
   'ObjectType': 'compute.AlarmSummary',
   'Warning': 0},
  'Ancestors': [],
  'AssetTag': 'Test test',
  'AvailableMemory': 196608,
  'BiosPostComplete': False,
  'ChassisId': '',
  'ClassId': 'compute.PhysicalSummary',
  'ConnectionStatus': '',
  'CpuCapacity': 44,
  'CreateTime': '2018-04-16T14:27:56.546Z',
  'DeviceMoId': '5ad4b2e337726e6c33c081b8',
  'Dn': 'sys/rack-unit-1',
  'DomainGroupMoid': '5b2541877a7662743465cc98',
  'FaultSummary': 65536,
  'Firmware': '3.1(2d)',
  'InventoryDeviceInfo': None,
  'Ipv4Address': '172.28.224.103',
  'KvmIpAddresses': [{'Address': '172.28.224.103',
    'Category': 'Equipment',
    'ClassId': 'compute.IpAddress',
    'DefaultGateway': '172.28.224.1',
    'Dn': 'sys/rack-unit-1/mgmt/if-1',
    'HttpPort': 80,
    'HttpsPort': 443,
    'KvmPort': 2068,
    'KvmVlan': 1,
    'Name': 'Outband',
    'O

## Step 7. See how many items were returned by `Results`
`Results` is a list which we can pass `response.json()['Results']` to the `len()` BIF (Built In Function) to see how many results are returned. The result tell us how many physical compute items are contained in the list.

In [12]:
len(response.json()['Results'])

129

## Step 8. Bringing it all together
Great! Now we see the number of physical compute devices and now we can pull more information from the returned JSON and organize it by Device, Chassis ID, Management Mode, Model, Memory, and CPU. The information in the `Management Mode` column shows if the phsyical compute device is managed in Intersight Management Mode.

> Intersight Managed Mode (IMM) is a new architecture that manages the UCS Fabric Interconnected systems through a Redfish-based standard model. If you are familiar with the UCS blades, it means the Fabric Interconnect is fully managed by Intersight. Instead of having the familiar UCSM (UCS Manager) interface available directly from the Fabric Interconnect, the interface and all of the Fabric Interconnect operations are managed by Intersight.

We do some CLI formatting to organize our data and see the type of compute hardware managed by Intersight along with its resources (memory and CPU). Then, we iterate over the JSON data and pull the data we're interested in. In this instance, the Model shows the UCS-X series hardware.

> Experiment! See if you can add more information to the list below by choosing other items from the returned JSON data above. For example, you could add a column with `NumCpuCores` and or `Ipv4Address` to the code below. There's no right or wrong answer as to which columns you would like displayed.

In [14]:
print ("{:<8} {:12} {:<21} {:<15} {:<10} {:<10}".format(
       'Device',
       'Chassis ID',
       'Management Mode',
       'Model',
        'Memory',
        'CPU'))
for num, items in enumerate(response.json()['Results'], start=1):
    print (
        "{:<8} {:<12} {:<21} {:<15} {:<10} {:<10}".format(
            num, 
            items['ChassisId'], 
            items['ManagementMode'], 
            items['Model'], 
            items['AvailableMemory'], 
            items['CpuCapacity']))

Device   Chassis ID   Management Mode       Model           Memory     CPU       
1                     IntersightStandalone  HXAF220C-M5SX   196608     44        
2                     IntersightStandalone  HXAF220C-M5SX   196608     44        
3                     IntersightStandalone  HXAF220C-M5SX   196608     44        
4                     IntersightStandalone  HXAF220C-M5SX   196608     44        
5                     IntersightStandalone  HXAF220C-M5SX   196608     44        
6                     IntersightStandalone  HXAF220C-M5SX   196608     44        
7                     IntersightStandalone  HX220C-M5SX     196608     44        
8                     IntersightStandalone  HX220C-M5SX     196608     0         
9                     IntersightStandalone  HXAF220C-M5SX   393216     67.2      
10                    IntersightStandalone  HXAF220C-M5SX   393216     67.2      
11                    IntersightStandalone  UCSC-C220-M5L   196608     62.399998 
12              

**Next Step: Provide Title of the Next Step**